## Definitions:

### Primitive root of unity
Any complex number that yields 1 when raised to some positive integer power n.
1. $ \zeta^n = 1 $
2. $ \zeta^k \neq 1 $ for all integers $ k $ such that $ 0 < k < n $.

$\zeta^n = e^{i(2\pi \frac{k}{n})}$

For 2N-th roots of unity, this means any number $\zeta$ that satisfies $\zeta^{2N}=1$

### Cyclotomic Polynomial
A **cyclotomic polynomial** is a special type of polynomial defined as the unique irreducible polynomial with integer coefficients whose roots are the **primitive roots of unity** $e^{i(2\pi\frac{k}{n})}$, where k runs over the positive integers less than n and coprime to n. The formula is equal to:

$$
\Phi_{n}(x) = \prod_{\substack{1 \leq k \leq n \\ \gcd(k, n) = 1}} \left(x - e^{2i\pi \frac{k}{n}}\right)
$$

**Special Case**: $\Phi_M(X) = X^N + 1$

The specific form $\Phi_M(X) = X^N + 1$ arises when:

- $M = 2N$, i.e., $M$ is an even number and M is a power of 2.
- The roots of $\Phi_{2N}(X)$ are the primitive $2N$-th roots of unity.


#### 2.1. Roots of $X^N + 1$

The equation $X^N + 1 = 0$ has $N$ distinct roots in the complex plane. These roots are the **odd powers** of $\xi$, a primitive $2N$-th root of unity:

$$
\xi = e^{2\pi i / (2N)}, \quad \text{so } \xi^{2k+1} \text{ for } k = 0, 1, \dots, N-1.
$$



#### 2.2. Factorization

The polynomial $X^N + 1$ factors as:

$$
X^N + 1 = \prod_{k=0}^{N-1} \left( X - \xi^{2k+1} \right),
$$

where $\xi^{2k+1}$ are the odd powers of the primitive $2N$-th root of unity $\xi$.



#### 2.3. Minimal Polynomial

When $M = 2N$, the primitive $2N$-th roots of unity are precisely the roots of $X^N + 1$. Hence, the minimal polynomial of the primitive $2N$-th roots of unity is:

$$
\Phi_{2N}(X) = X^N + 1.
$$


### Isomorphism

An isomorphism between two algebraic structures is a bijective homomorphism.

**Addition Preservation**:
$\sigma(m1 + m2) = \sigma(m1) + \sigma(m2)$

**Multiplication Preservation**:
$\sigma(m1 * m2) = \sigma(m1) \circ \sigma(m2)$

- Injectivity: No two distinct polynomials map to the same vector under σ.
- Surjectivity: Every vector in ${C^N}$ (satisfying the necessary properties) is the image of some polynomial under σσ.




## Plaintext Encoding and Decoding

### Canonical embedding $\sigma:$

### Encoding

For encoding, we aim to project a complex vector into the subspace:

$$
H = \{(z_j)_{j \in \mathbb{Z}^*_M} : z_j = z_{-j}\}.
$$

This is the $\pi^{-1}$ projection discussed in the original paper, where half of the entries are conjugates of the other half. We know that:

$$
\sigma(\mathcal{R}) \subseteq \mathbb{H} = \{ z \in \mathbb{C}^N : z_j = z_{-j} \}.
$$

The process involves the following steps:

1. **Projection using $\pi^{-1}$:**  
   The scaled vector is then projected using $\pi^{-1}$. This ensures that the resulting vector lies in $\mathbb{H}$.
   


In [300]:
# code inspired from https://colab.research.google.com/drive/1cdue90Fg_EB5cxxTYcv2_8_XxQnpnVWg?usp=sharing#scrollTo=qsDuOVro1H15

from math import sin,cos,pi
import numpy as np
from numpy.polynomial import Polynomial

In [301]:
def pi_inverse(z: np.array) -> np.array:
    """Expands a vector of C^{N/2} by expanding it with its
    complex conjugate."""
    
    z_conjugate = z[::-1]
    z_conjugate = [np.conjugate(x) for x in z_conjugate]
    return np.concatenate([z, z_conjugate])

# TOY EXAMPLE FROM https://eprint.iacr.org/2016/421.pdf section 3.2

M = 8 # parameter from the paper
delta = 64 # scaling factor 
xi = np.exp(2 * np.pi * 1j / M) # Mth root of unity which will be used as a basis for our computation
z = np.array([3 +4j, 2 - 1j]) # vector to encode
pi_z = pi_inverse(z) # pi^-1
pi_z

array([3.+4.j, 2.-1.j, 2.+1.j, 3.-4.j])

2. **Multiplication with $\Delta$:**  
   We start by multiplying the vector with $\Delta$ to scale it appropriately.

In [302]:
scaled_pi_z = delta * pi_z
scaled_pi_z # scaled vector

array([192.+256.j, 128. -64.j, 128. +64.j, 192.-256.j])

3. **Coordinate-Wise Random Rounding:**  
   To project onto $\sigma(\mathcal{R})$, we use a technique called *coordinate-wise random rounding*. This is a method to discretize complex numbers in each coordinate independently while preserving certain statistical properties. For details, refer to [this paper](https://web.eecs.umich.edu/~cpeikert/pubs/toolkit.pdf).

    
   A. **Orthogonal $\mathbb{Z}$-Basis:**
   - The ring $\mathcal{R}$ has an orthogonal $\mathbb{Z}$-basis $\{1, X, \ldots, X^{N-1}\}$.
   - Since $\sigma$ is an isomorphism, $\sigma(\mathcal{R})$ has a corresponding orthogonal $\mathbb{Z}$-basis:
     $$
     \beta = \{b_1, b_2, \ldots, b_N\} = \{\sigma(1), \sigma(X), \ldots, \sigma(X^{N-1})\}.
     $$

In [303]:
def vandermonde(xi: np.complex128, M: int) -> np.array:
    """Computes the Vandermonde matrix from a m-th root of unity."""
    
    N = M //2
    matrix = []
    # We will generate each row of the matrix
    for i in range(N):
        # For each row we select a different root
        root = xi ** (2 * i + 1)
        row = []

        # Then we store its powers
        for j in range(N):
            row.append(root ** j)
        matrix.append(row)
    return matrix

def create_sigma_R_basis(xi, M):
    """Creates the basis (sigma(1), sigma(X), ..., sigma(X** N-1))."""
    return np.array(vandermonde(xi, M)).T
    
sigma_R_basis = create_sigma_R_basis(xi, M)

We can now have a look at the basis $\{\sigma(1), \sigma(X), \ldots, \sigma(X^{N-1})\}$.

In [304]:
sigma_R_basis

array([[ 1.00000000e+00+0.j        ,  1.00000000e+00+0.j        ,
         1.00000000e+00+0.j        ,  1.00000000e+00+0.j        ],
       [ 7.07106781e-01+0.70710678j, -7.07106781e-01+0.70710678j,
        -7.07106781e-01-0.70710678j,  7.07106781e-01-0.70710678j],
       [ 1.79380389e-16+1.j        , -4.67705010e-16-1.j        ,
         7.70954637e-16+1.j        , -1.11479041e-15-1.j        ],
       [-7.07106781e-01+0.70710678j,  7.07106781e-01+0.70710678j,
         7.07106781e-01-0.70710678j, -7.07106781e-01-0.70710678j]])

B. **Projection in the Basis:**
   - For any $z \in \mathbb{H}$, we can write:
     $$
     z = \sum_{i=1}^N z_i b_i,
     $$
     where $z_i$ are the coordinates of $z$ in the basis $\beta$. These coordinates are computed as:
     $$
     z_i = \frac{\langle z, b_i \rangle}{\|b_i\|^2} \in \mathbb{R}.
     $$
    - Here, $\langle x, y \rangle$ is the **Hermitian inner product**:
    $$
    \langle x, y \rangle = \sum_{k=1}^N x_k \overline{y_k},
    $$
    where $\overline{y_k}$ denotes the complex conjugate of $y_k$.

In [331]:
def compute_basis_coordinates(z):
    """Computes the coordinates of a vector with respect to the orthogonal lattice basis."""
    output = np.array([np.real(np.vdot(z, b) / np.vdot(b,b)) for b in sigma_R_basis])
    return output


coordinates = compute_basis_coordinates(scaled_pi_z)

C. **Rounding to Integers:**
   - Since the coordinates $z_i$ are real numbers (not integers), we apply **coordinate-wise random rounding**:
     - Each $z_i$ is rounded to either $\lfloor z_i \rfloor$ or $\lfloor z_i \rfloor + 1$.
     - The probability of rounding to $\lfloor z_i \rfloor + 1$ is proportional to the fractional part of $z_i$.

In [306]:
def round_coordinates(coordinates):
    """Gives the integral rest."""
    coordinates = coordinates - np.floor(coordinates)
    return coordinates

def coordinate_wise_random_rounding(coordinates):
    """Rounds coordinates randonmly."""
    r = round_coordinates(coordinates)
    f = np.array([np.random.choice([c, c-1], 1, p=[1-c, c]) for c in r]).reshape(-1)
    
    rounded_coordinates = coordinates - f
    rounded_coordinates = [int(coeff) for coeff in rounded_coordinates]
    return rounded_coordinates

rounded_coordinates = coordinate_wise_random_rounding(coordinates)
rounded_coordinates

[160, 90, 160, 45]

D. **Reconstruction of $z$:**
   - Once we have the rounded coordinates $\{z_i\}$, we reconstruct $z$ using the basis $\beta$:
     $$
     z = \sum_{i=1}^N z_i b_i.
     $$


In [307]:
y = np.matmul(sigma_R_basis.T, rounded_coordinates)
y

array([191.81980515+255.45941546j, 128.18019485 -64.54058454j,
       128.18019485 +64.54058454j, 191.81980515-255.45941546j])

4. **Applying $\sigma^{-1}$:**  
Finally, we apply $\sigma^{-1}$, as described below. This step outputs an element in $\mathbb{R}$, completing the encoding process.

Find a polynomial $m(X) = \sum_{i=0}^{N-1} \alpha_i X^i \in \mathbb{C}[X]/(X^N + 1)$, given a vector $z \in \mathbb{C}^N$, such that $\sigma(m) = (m(\xi), m(\xi^3), \ldots, m(\xi^{2N-1})) = (z_1, \ldots, z_N)$ where $\xi_M$, the $M$-th root of unity: $\xi_M = e^{2i\pi/M}$. So we get:
$\sum_{j=0}^{N-1} \alpha_j (\xi^{2i-1})^j = z_i, \; i = 1, \ldots, N.$

This can be viewed as a linear equation:

$A\alpha = z$, with $A$ the Vandermonde matrix of the $(\xi^{2i-1})_{i=1,\ldots,N}$, $\alpha$ the vector of the polynomial coefficients, and $z$ the vector we want to encode.

$$
\begin{bmatrix}
1 & \xi_1 & \xi_1^2 & \xi_1^3 \\
1 & \xi_3 & \xi_3^2 & \xi_3^3 \\
1 & \xi_5 & \xi_5^2 & \xi_5^3 \\
1 & \xi_7 & \xi_7^2 & \xi_7^3
\end{bmatrix}
\cdot
\begin{bmatrix}
a_1 \\
a_2 \\
a_3 \\
a_4
\end{bmatrix}
=\begin{bmatrix}
z_1 \\
z_2 \\
z_3 \\
z_4
\end{bmatrix}
$$

where $\xi_1 = \xi_M, \xi_3 = (\xi_M)^3, \xi_5 = (\xi_M)^5, \ldots$.



In [308]:
def sigma_inverse(b: np.array) -> Polynomial:
    """Encodes the vector b in a polynomial using an M-th root of unity."""

    # First we create the Vandermonde matrix
    A = vandermonde(xi, M)

    # Then we solve the system
    coeffs = np.linalg.solve(A, b)

    # Finally we output the polynomial
    p = Polynomial(coeffs)
    
    # We round it afterwards due to numerical imprecision
    coef = np.round(np.real(p.coef)).astype(int)
    return Polynomial(coef)

encoded = sigma_inverse(y)
encoded

Polynomial([160.,  90., 160.,  45.], domain=[-1.,  1.], window=[-1.,  1.], symbol='x')

Therefore, we have that:

$\alpha = A^{-1}z$, and that $\sigma^{-1}(z) = \sum_{i=0}^{N-1} \alpha_i X^i \in \mathbb{C}[X]/(X^N + 1)$. 

### Decoding

For decoding, we work with a cyclotomic polynomial ring $\mathbb{Z}[X]/(X^N + 1)$ and aim to decode it to a vector $z \in \mathbb{C}^N$ such that:
$$
z = \pi \circ \sigma(\Delta^{-1} * m) \in \mathbb{C}^{N/2}.
$$

The decoding process involves the following steps:


1. **Scaling by $1 / \Delta$:**  
   Multiply the polynomial $m$ by the inverse of $\Delta$, i.e., $1 / \Delta$. This rescales the polynomial back to the correct domain.


In [309]:
rescaled_p = encoded / delta

2. **Applying $\sigma$:**  
   Next, apply $\sigma$, which maps the polynomial from $\mathbb{Z}[X]/(X^N + 1)$ to a complex vector in $\mathbb{C}^N$. This is explained in detail below.

To decode a polynomial $m(X)$ into a vector $z$, we evaluate on certain values, which will be the roots of $\Phi(M) = X^N + 1$, where the $N$ roots are $\xi, \xi^3, \ldots, \xi^{2N-1}$. 


In [310]:
def sigma(p: Polynomial) -> np.array:
    """Decodes a polynomial by applying it to the M-th roots of unity."""

    outputs = []
    N = M //2

    # We simply apply the polynomial on the roots
    for i in range(N):
        root = xi ** (2 * i + 1)
        output = p(root)
        outputs.append(output)
    return np.array(outputs)

z_vector = sigma(rescaled_p)

3. **Projection to $\mathbb{C}^{N/2}$:**  
   The final step is to project the vector onto $\mathbb{C}^{N/2}$. Since the structure ensures that the second half of the array is the conjugate of the first half, this projection simply involves returning the first half of the array.

In [312]:
def pi(z: np.array) -> np.array:
    """Projects a vector of H into C^{N/2}."""
    
    N = M // 4
    return z[:N]

decoded = pi(z_vector)
decoded

array([2.99718446+3.99155337j, 2.00281554-1.00844663j])

If we put everything together we have:

In [330]:
def sigma_R_discretization(z):
    """Projects a vector on the lattice using coordinate wise random rounding."""
    coordinates = compute_basis_coordinates(z)
    
    rounded_coordinates = coordinate_wise_random_rounding(coordinates)
    y = np.matmul(sigma_R_basis.T, rounded_coordinates)
    return y

def decode(p: Polynomial) -> np.array:
        """Decodes a polynomial by removing the scale, 
        evaluating on the roots, and project it on C^(N/2)"""
        rescaled_p = p / scale
        z = sigma(rescaled_p)
        pi_z = pi(z)
        return pi_z

def encode(z: np.array) -> Polynomial:
    """Encodes a vector by expanding it first to H,
    scale it, project it on the lattice of sigma(R), and performs
    sigma inverse.
    """
    pi_z = pi_inverse(z)
    scaled_pi_z = scale * pi_z
    rounded_scale_pi_zi = sigma_R_discretization(scaled_pi_z)
    p = sigma_inverse(rounded_scale_pi_zi)
    
    # We round it afterwards due to numerical imprecision
    coef = np.round(np.real(p.coef)).astype(int)
    p = Polynomial(coef)
    return p


# TOY EXAMPLE FROM https://eprint.iacr.org/2016/421.pdf section 3.2

encoded_pol = encode(z)
decoded = decode(encoded_pol)

print(f"Original vector: ", z)
print(f"Vector after encoding and decoding: ", decoded)


Original vector:  [3.+4.j 2.-1.j]
Vector after encoding and decoding:  [2.99718446+3.99155337j 2.00281554-1.00844663j]


In [9]:
# code from https://github.com/AI-Tech-Research-Lab/Introduction-to-BFV-HE-ML/blob/main/BFV_theory/BFV_theory.ipynb

def Poly(coeffs):
    """
    Helper function to build polynomials, passing a dictionary of coefficients.
    For example, passing {0: 1, 1: -2, 2: 2} returns the polynomial
    2X**2 - 2X + 1
    """
    max_power = max(coeffs.keys())
    _coeffs = np.zeros(max_power + 1)
     
    for i, c in coeffs.items():
        _coeffs[i] = c
        
    return Polynomial(_coeffs)

def pr(p):
    """ 
    Helper function to pretty-print the polynomials, with human order
    of powers, removed trailing .0, etc.
    """
    coefs = p.coef
    res = ""
    
    powers = range(len(coefs)-1, -1, -1)
    for power, coeff in zip(powers, reversed(coefs)):
        if coeff == 0:
            continue
        
        if int(coeff) == coeff:
            coeff = int(coeff)
                  
        sign = "- " if coeff < 0 else "+ "
        
        if power == 0:
            value = abs(coeff)
        elif abs(coeff) != 1:
            value = abs(coeff)
        else:
            value = ""

        power_sign = {0: "", 1: "X"}
        def_power_sign = f"X**{power}"
        
        res += f" {sign}{value}{power_sign.get(power, def_power_sign)}"
        
    if res[1] == "+":
        return res[3:]
    if res[1] == "-":
        return res[1:]

def mod_on_coefficients(polynomial, modulo):
    """
    Apply the modulo on the coefficients of a polynomial.
    """
    coefs = polynomial.coef
    mod_coefs = []
    for c in coefs:
        mod_coefs.append(c % modulo)
        
    return Polynomial(mod_coefs)

def round_on_nearest_integer(polynomial):
    """
    Round the coefficients of a polynomial to the
    nearest integer.
    """
    coefs = polynomial.coef
    round_coefs = []
    for c in coefs:
        round_coefs.append(round(c))
    
    return Polynomial(round_coefs)


def reduce_polynomial(polynomial, cyclotomic_polynomial, q):
    """
    Reduce a polynomial modulo a cyclotomic polynomial and a modulus.
    """
    remainder = divmod(polynomial, polynomial_modulus)[1] # polynomial modulo cyclotomic polynomial -> get the remainder
    return mod_on_coefficients(remainder, q) # perfrom mod operation on coefficients


In [10]:
import random

def secret_key_pol(M, h):

    s = {}
    non_zero_indices = random.sample(range(M), h)
    
    # Assign ±1 randomly to the selected positions
    for idx in non_zero_indices:
        s[idx] = random.choice([-1, 1])
    
    return Poly(s)

def create_error_pol(M, sigma, mu):
    e = np.random.normal(mu, sigma, M)
    e_map = {}
    for i in range(len(e)):
        e_map[i] = round(e[i])
    
    return Poly(e_map)

def sample_pol(M, modulous):
    coeffs = {}
    for i in range(M):
        c = np.random.randint(0, modulous)
        coeffs[i] = c
    return Poly(coeffs)
    

In [171]:
delta = 64
M = 32
N = M // 2 # degree of the polynomial, also a power of 2
polynomial_modulus = Poly({0: 1, N: 1})
h = 5 # the number of non zero coefficients
L = 1 # ciphertext level -> decreases as the computation progresses
q0 = delta << 2 # base modulous, q
qL = q0 * delta**L

mu = 0
sigma = 3.2
p = 997


s = secret_key_pol(N, h) # secret key
a = sample_pol(N, qL)
e = create_error_pol(N, sigma, mu)

b = (-a*s)+e
b = reduce_polynomial(b, polynomial_modulus, qL)

# public key
pk = (b, a)

# Evaluation key generation
a_prime = sample_pol(N, qL*p)
e_prime = create_error_pol(N, sigma, mu)

b_prime = (-a_prime*s)+e_prime+p*(s**2)
b_prime = reduce_polynomial(b_prime, polynomial_modulus, qL*p)

eval_key = (b_prime, a_prime)

In [193]:
e1 = create_error_pol(N, sigma, mu)
e2 = create_error_pol(N, sigma, mu)
coeffs = {}

for i in range(0, N):
    coeffs[i] = np.random.randint(-1, 2)    
v = Poly(coeffs)

print(f"e_1: {pr(e1)}\n")
print(f"e_2: {pr(e2)}\n")
print(f"v: {pr(v)}\n")

e_1: X**15 + 2X**14 - 9X**12 - 8X**11 + 2X**10 + 4X**8 - X**7 - 4X**6 + 2X**5 - X**4 - 6X**3 + 2X**2 + 2X - 3

e_2: - 3X**15 + X**14 + 4X**13 + 7X**12 + X**11 + 2X**10 + 2X**9 - 2X**8 - 5X**7 - 3X**6 - 3X**4 + X**2 - 2X - 3

v: - X**15 - X**12 + X**11 - X**10 + X**9 + X**8 - X**7 - X**4 - X**2 - 1



In [209]:
from math import floor

m = Poly({0: 160, 1: 91, 2: 160, 3:45})

ct0 = pk[0] * v + e1 + floor((qL/p)) * m
ct0 = divmod(ct0, polynomial_modulus)[1]
ct0 = mod_on_coefficients(ct0, qL)

ct1 = pk[1] * v + e2
ct1 = divmod(ct1, polynomial_modulus)[1]
ct1 = mod_on_coefficients(ct1, qL)

ciphertext = (ct0, ct1)
print(pr(ct0), pr(ct1))

63674554X**15 + 31439884X**14 + 52592614X**13 + 48757533X**12 + 66218841X**11 + 38337549X**10 + 24968707X**9 + 20659940X**8 + 40942671X**7 + 24261738X**6 + 14589591X**5 + 33039332X**4 + 63414118X**3 + 18745556X**2 + 43414306X + 1843027 43045451X**15 + 34966882X**14 + 64537927X**13 + 54621727X**12 + 59145777X**11 + 50294983X**10 + 59849089X**9 + 62158948X**8 + 65817519X**7 + 21379375X**6 + 32857074X**5 + 66347533X**4 + 21395355X**3 + 50150093X**2 + 38060665X + 16854629


In [210]:
plaintext = ciphertext[1] * s + ciphertext[0]
plaintext = divmod(plaintext, polynomial_modulus)[1]
plaintext = mod_on_coefficients(plaintext, qL)
plaintext = plaintext * p/qL
plaintext = round_on_nearest_integer(plaintext)
plaintext = mod_on_coefficients(plaintext, p)

print(f"Plain starting message: {pr(m)}")
print(f"Final decryption result: {pr(plaintext)}")

Plain starting message: 45X**3 + 160X**2 + 91X + 160
Final decryption result: 45X**3 + 160X**2 + 91X + 160
